# Answering Business Questions using SQL

On this project we will make some analysis using SQL to get some vision about our business our database called " chinook.db " let us start by connect our data base by the following code

In [1]:
import sqlalchemy

In [2]:

sqlalchemy.create_engine("sqlite:///chinook.db")
%load_ext sql
%sql sqlite:///chinook.db

'Connected: @chinook.db'

In [3]:
%%sql
SELECT 
    name, 
    type
FROM sqlite_master
WHERE type IN ("table", "view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


I will need to write a query to find out which genres sell the most tracks in the USA and write up a summary of findings.

In [4]:
%%sql
WITH total_sold AS (
SELECT COUNT(*) tracks_sold
  FROM customer c
 INNER JOIN invoice i ON i.customer_id = c.customer_id AND c.country = 'USA'
 INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
)

SELECT g.name genre, 
       COUNT(*) tracks_sold, 
       COUNT(*) * 1.0 / (SELECT COUNT(*) tracks_sold
                         FROM customer c 
                         INNER JOIN invoice i ON i.customer_id = c.customer_id AND c.country = 'USA'
                         INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id) percentage
FROM customer c
INNER JOIN invoice i ON i.customer_id = c.customer_id AND c.country = 'USA'
INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY genre
ORDER BY 2 DESC;


 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


According to our observe we can see that 3 genres: Rock, Alternative & Punk, Metal are the most popular in USA. Moreover we should notice that Rock genre alone accounts for **53%** of sales

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. I have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [5]:
%%sql
    SELECT e.first_name || " " || e.last_name employee, ROUND(SUM(i.total), 2) total_sales
      FROM customer c
INNER JOIN invoice i ON i.customer_id = c.customer_id
INNER JOIN employee e ON e.employee_id = c.support_rep_id
  GROUP BY 1
  ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


employee,total_sales
Jane Peacock,1731.51
Margaret Park,1584.0
Steve Johnson,1393.92


According to analyze the purchases of customers belonging to each employee we see Jane Peacock is the top and there is 20% difference between top and bottom employees.

Our next task is to write a query that collects data on purchases from different countries and where a country has only one customer, collect them into an "Other" group.
<br>
The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.
<br>
For each country we should include:
<br>
total number of customers
<br>
total value of sales
<br>
average value of sales per customer
<br>
average order value

In [10]:
%%sql
WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT COUNT(*)
                 FROM customer
                 WHERE country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers total_num_of_customers,
    total_sales,
    average_order,
    sales_avg_customer
FROM
    (
    SELECT
        country,
        count(DISTINCT customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / COUNT(DISTINCT customer_id) sales_avg_customer,
        SUM(unit_price) / COUNT(DISTINCT invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

 * sqlite:///chinook.db
Done.


country,total_num_of_customers,total_sales,average_order,sales_avg_customer
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


FROM above we find that:

- USA have the maximum of our sales with total sales 1040 taking about 22% of our sales , but we have about 13 customers so our sales average per customer about 80 with sales average per order 7.94.
- Czech Republic have the Maximum sales average per customer 136.62 and by order 9.11 although the it take only 6% of our sales , so we can increase our sales on it by increasing number of customers
- also on United Kingdom, Portugal & India we have an opportunity to increasing our customers by increasing our sales there as sales average per customer & average order also is better than USA except average order in portugal.

In [13]:
%%sql 
WITH 
    invoice_first_track AS 
    (
        SELECT 
            il.invoice_id,
            MIN(il.track_id) first_track_id
        FROM invoice_line il 
        GROUP BY 1
    )
SELECT 
        album_purchase,
        COUNT(invoice_id) number_of_invoices,
        CAST(COUNT(invoice_id) AS FLOAT) / (
                                                SELECT COUNT(*) FROM invoice
                                          ) percent
FROM (
        SELECT 
            ift.*,
            CASE 
                WHEN (
                        SELECT t.track_id FROM track t
                        WHERE t.album_id = (
                                                SELECT t2.album_id FROM track t2
                                                WHERE t2.track_id = ift.first_track_id
                                           )
                        EXCEPT 
                        
                        SELECT il2.track_id FROM invoice_line il2
                        WHERE il2.invoice_id = ift.invoice_id
                     ) IS NULL
                AND 
                    (
                        SELECT il2.track_id FROM invoice_line il2
                        WHERE il2.invoice_id = ift.invoice_id
                        
                        EXCEPT 
                        
                        SELECT t.track_id FROM track t 
                        WHERE t.album_id = (
                                                SELECT t2.album_id FROM track t2
                                                WHERE t2.track_id = ift.first_track_id
                                           )
                    
                    ) IS NULL 
                THEN 'yes'
                ELSE 'no'
            END AS 'album_purchase'
        FROM invoice_first_track ift
     )
GROUP BY album_purchase;

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Also we can see purchase depends on invidual tracks more the album by about 62.8%.

Let's find which artist is used in the most playlists?

In [14]:
%%sql 
WITH 
    playlist_artist AS 
    (
        SELECT 
            pt.playlist_id,
            pt.track_id,
            t.album_id,
            al.artist_id
        FROM playlist_track pt 
        INNER JOIN track t ON t.track_id = pt.track_id
        INNER JOIN album al ON al.album_id = t.album_id
        ORDER BY al.artist_id
    )
SELECT 
    ar.artist_id,
    ar.name artist_name,
    COUNT(plar.track_id) most_artist
FROM artist ar 
INNER JOIN playlist_artist plar ON plar.artist_id = ar.artist_id
GROUP BY ar.artist_id
ORDER BY 3 DESC,2
LIMIT 5;

 * sqlite:///chinook.db
Done.


artist_id,artist_name,most_artist
90,Iron Maiden,516
150,U2,333
50,Metallica,296
22,Led Zeppelin,252
58,Deep Purple,226


From table above we can explore that artist in most of the playlist is Iron Maiden. He is in playlists with 516 tracks, but be careful that same track may be found on different playlists.

So now let us to move to another question:
How many tracks were purchased vs not purchased?

In [15]:
%%sql 
WITH 
    artist_album_tracks AS 
    (
        SELECT
            al.artist_id,
             t.album_id,
            t.track_id        
        FROM track t 
        INNER JOIN album al ON al.album_id = t.album_id  
        ORDER BY 1,2
    ),
    purchased_tracks AS 
    (
        SELECT 
            DISTINCT(t.track_id) track_id
            FROM track t 
            INNER JOIN invoice_line il ON il.track_id = t.track_id
            ORDER BY 1
    ),
    tracks_purchasing_category
    AS(
        SELECT 
            aralt.*,
            CASE 
                WHEN (
                        SELECT t.track_id FROM track t
                        WHERE  t.track_id = aralt.track_id
                        EXCEPT 
                        SELECT pt.track_id FROM purchased_tracks pt
                     ) IS NULL 
                THEN 'yes'
                ELSE 'no'
            END AS 'purchased_track'
        FROM artist_album_tracks aralt
    )

SELECT 
    tpc.purchased_track,
    COUNT(tpc.track_id) number_of_tracks,
    ROUND(CAST(COUNT(tpc.track_id) AS FLOAT)/(SELECT CAST(COUNT(tpc.track_id) AS FLOAT) FROM tracks_purchasing_category tpc),2) 'category%'
FROM tracks_purchasing_category tpc
GROUP BY 1
ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


purchased_track,number_of_tracks,category%
yes,1806,0.52
no,1697,0.48


From above we can find that 52% of tracks have been purchased and 48% of tracks not purchased.

Let's now answer to the next question:
<strong>Do protected vs non-protected media types have an effect on popularity?</strong>

In [16]:
%%sql
CREATE VIEW protected_status AS
SELECT * ,
    CASE 
        WHEN
            name LIKE '%Protected%' 
                THEN 'protected'
        ELSE
            'unprotected'
    END AS status
FROM media_type

 * sqlite:///chinook.db
Done.


[]

In [17]:
%%sql
SELECT * FROM protected_status

 * sqlite:///chinook.db
Done.


media_type_id,name,status
1,MPEG audio file,unprotected
2,Protected AAC audio file,protected
3,Protected MPEG-4 video file,protected
4,Purchased AAC audio file,unprotected
5,AAC audio file,unprotected


In [18]:
%%sql
WITH 
protected_status_applied AS
(
    SELECT 
        il.track_id,
        ps.status
    FROM invoice_line AS il
    INNER JOIN track AS t on t.track_id=il.track_id
    INNER JOIN protected_status AS ps ON ps.media_type_id=t.media_type_id
)

SELECT 
    status,
    COUNT(*) AS count,
    ROUND(CAST(COUNT(*) AS FLOAT)*100
          / (SELECT COUNT(*) FROM protected_status_applied)
          ,0) AS 'percentage_sold_based_on_media_type'

FROM protected_status_applied AS psa
GROUP BY status

 * sqlite:///chinook.db
Done.


status,count,percentage_sold_based_on_media_type
protected,442,9.0
unprotected,4315,91.0


Yes, popularity is affected by media type. 91% of sale is of unprotected media.